# Computational Theory Assignment

In [ ]:
# Necessary imports

import numpy as np

## Problem 1

### SHA-1 Parity Function

The `Parity` function implements the SHA-1 Parity operation, which performs a bitwise XOR across three input values. This is commonly used in cryptographic hash functions to mix bits and increase unpredictability.

Each input should be an integer representing a binary value. The function returns the result as a binary string.

#### Example Usage

```python
result = Parity(0b1111, 0b1010, 0b0101)
print(result)  # Output: '0b0'
```

In this example, the function computes the XOR of the three binary numbers and returns the result in binary format.

#### Code

In [ ]:
def Parity(x, y, z):
    """
    SHA-1 Parity function.

    Parameters:
        x (int): first value (interpreted as binary).
        y (int): second value (interpreted as binary).
        z (int): third value (interpreted as binary).

    Returns:
        str: The result of the parity as a binary string.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    # SHA-1 Parity function = bitwise XOR of each bit position
    result = x ^ y ^ z   # ^ is the bitwise XOR operator in Python

    return bin(result)


#### Testing

In [ ]:
def test_parity():
    assert Parity(0b0, 0b0, 0b0) == '0b0', "All zeros should return 0b0"
    assert Parity(0b1, 0b0, 0b0) == '0b1', "Single one should return 0b1"
    assert Parity(0b1, 0b1, 0b0) == '0b0', "Two ones should return 0b0"
    assert Parity(0b1, 0b1, 0b1) == '0b1', "Three ones should return 0b1"
    assert Parity(0b101, 0b010, 0b001) == '0b110', "Mixed bits test"
    assert Parity(0b1111, 0b1010, 0b0101) == '0b0', "4-bit test"
    print('All test cases passed.')

In [ ]:
# Run all tests
test_parity()

All test cases passed.


### SHA-1 Choose (Ch) Function

The `Ch` function implements the SHA-1 'choose' operation, which selects bits from the second or third input based on the bits of the first input. For each bit position, if the corresponding bit in `x` is 1, the result is the bit from `y`; otherwise, it's the bit from `z`. This is expressed as `(x & y) ^ (~x & z)`.

Each input should be an integer representing a binary value. The function returns the result as a binary string.

#### Example Usage

```python
result = Ch(0b1111, 0b1010, 0b0101)
print(result)  # Output: '0b1010'
```

In this example, the function chooses bits from `y` where `x` is 1, and from `z` where `x` is 0.

In [ ]:
def Ch(x, y, z):
    """
    SHA-1 choose (Ch) function.

    Parameters:
        x (int): first value (interpreted as binary).
        y (int): second value (interpreted as binary).
        z (int): third value (interpreted as binary).

    Returns:
        str: The result of the choose function as a binary string.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    # SHA-1 choose function: (x & y) ^ (~x & z)
    result = (x & y) ^ (~x & z)

    return bin(result)

In [ ]:
def test_Ch():
    assert Ch(0b0, 0b0, 0b0) == '0b0', "All zeros should return 0b0"
    assert Ch(0b1, 0b0, 0b0) == '0b0', "x=1, y=0, z=0 should return 0b0"
    assert Ch(0b1, 0b1, 0b0) == '0b1', "x=1, y=1, z=0 should return 0b1"
    assert Ch(0b0, 0b1, 0b1) == '0b1', "x=0, y=1, z=1 should return 0b1"
    assert Ch(0b101, 0b010, 0b001) == '0b001', "Mixed bits test"
    assert Ch(0b1111, 0b1010, 0b0101) == '0b1010', "4-bit test"
    print('All test cases passed.')

In [ ]:
# Run all tests
test_Ch()

All test cases passed.


### SHA-1 Majority (Maj) Function

The `Maj` function implements the SHA-1 'majority' operation, which returns the majority value of each bit position among three inputs. For each bit, if at least two of the three inputs have a 1, the result is 1; otherwise, it's 0. This is expressed as `(x & y) ^ (x & z) ^ (y & z)`.

Each input should be an integer representing a binary value. The function returns the result as a binary string.

#### Example Usage

```python
result = Maj(0b1111, 0b1010, 0b0101)
print(result)  # Output: '0b1111'
```

In this example, the function computes the majority for each bit position and returns the result in binary format.

#### Code

In [ ]:
def Maj(x, y, z):
    """
    SHA-1 majority (Maj) function.

    Parameters:
        x (int): first value (interpreted as binary).
        y (int): second value (interpreted as binary).
        z (int): third value (interpreted as binary).

    Returns:
        str: The result of the majority function as a binary string.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    # SHA-1 majority function: (x & y) ^ (x & z) ^ (y & z)
    result = (x & y) ^ (x & z) ^ (y & z)

    return bin(result)

#### Testing

In [ ]:
def test_Maj():
    assert Maj(0b0, 0b0, 0b0) == '0b0', "All zeros should return 0b0"
    assert Maj(0b1, 0b0, 0b0) == '0b0', "Only one bit set should return 0b0"
    assert Maj(0b1, 0b1, 0b0) == '0b1', "Two bits set should return 0b1"
    assert Maj(0b1, 0b1, 0b1) == '0b1', "All bits set should return 0b1"
    assert Maj(0b101, 0b010, 0b001) == '0b1', "Mixed bits test"
    assert Maj(0b1111, 0b1010, 0b0101) == '0b1111', "4-bit test"
    assert Maj(0b111, 0b110, 0b111) == '0b111', "Example of full bits"
    print('All test cases passed.')

In [ ]:
# Run all tests
test_Maj()

All test cases passed.


## Problem 2

## Problem 3

## Problem 4

## Problem 5

## End